# 第二次检测修复字符串

In [7]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
original_language = "English"
target_language = "Vietnamese"
def generate_text(index, data):
    try:
        if str(data['备注']).strip() == 'SPECIAL VALUE' or str(data['备注']).strip() == 'SPECIAL VALUE, INCLUDE UPPERCASE LETTER' or str(data['备注']).strip() == '':
            return index, str(data['待翻译(译)']).strip()
        if str(data['参考语言(英文)']).strip().lower() != str(data['简体中文(源)']).strip().lower() and str(data['参考语言(英文)']).strip().lower() == str(data['待翻译(译)']).strip().lower():
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": f"You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}. Preserving its formatting without adding extra content."},
                    {"role": "user", "content": str(data['参考语言(英文)'])}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
        if str(data['参考语言(英文)']).strip().lower() == str(data['简体中文(源)']).strip().lower():
            return index, str(data['参考语言(英文)']).strip()
        else:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": "Translate English: The received L/G %s is not submitted \nTo Spanish: El L/G recibido no se ha enviado. \nPlease fix the translate error, because %s is missing, translate it directly and without adding any extra content."},
                    {"role": "assistant", "content": "El L/G recibido %s no se ha enviado."},
                    {"role": "user", "content": f"Translate {original_language}: {str(data['参考语言(英文)'])} \nTo {target_language}: {str(data['待翻译(译)'])} \n Please fix the translate error, Because {str(data['备注'])}. translate it directly and without adding any extra content."}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
    except:
        return index, str(data['待翻译(译)']).strip()

In [8]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, f'待翻译(译)'] = output

to_fix['备注'] = ""
# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越_processed.xlsx", index=False)

100%|██████████| 124202/124202 [02:26<00:00, 849.76it/s] 


# 文件合并

In [2]:
import pandas as pd

original = "英"
target = "印尼"
# 定义要合并的 Excel 文件列表
excel_files = [
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_1_10000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_10001_20000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_20001_30000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_30001_40000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_40001_50000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_50001_60000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_60001_70000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_70001_80000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_80001_90000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_90001_100000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_100001_110000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_110001_120000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_120001_130000.xlsx",
]

# 读取并合并所有 Excel 文件
df_list = [pd.read_excel(file) for file in excel_files]
merged_df = pd.concat(df_list, ignore_index=True)

# 保存合并后的 DataFrame 到新的 Excel 文件
merged_df.to_excel(f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}.xlsx", index=False)


# 友互通翻译

In [34]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
target_column = "缅甸语"

def generate_text(index, data):
    if not pd.isnull(data[target_column]):
        return index, data[target_column]
    completion = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "user", "content": f"Translate English to Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['简体中文(源)'])},
            {"role": "user", "content": f"Translate English to Traditional Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['繁体中文（译）'])},
            {"role": "user", "content": f"Translate English to {target_column}: {str(data['English(译)'])}, please directly translate it and do not output any extra content"}
        ],
        temperature=0
    )
    return index, completion.choices[0].message.content

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, target_column] = output

# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx", index=False)

  0%|          | 0/1832 [00:00<?, ?it/s]C:\Users\wangz\AppData\Local\Temp\ipykernel_27064\1630559578.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'အထောက်အထားဓာတ်ပုံတင်ပါ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
100%|██████████| 1832/1832 [00:18<00:00, 100.40it/s]


In [2]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/", api_key="")
target_column = "印尼语"

def generate_text(index, data):
    if not pd.isnull(data[target_column]):
        return index, data[target_column]
    completion = client.chat.completions.create(
        model='gemini-1.5-flash',
        messages=[
            {"role": "user", "content": f"Translate English to Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['简体中文(源)'])},
            {"role": "user", "content": f"Translate English to Traditional Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['繁体中文（译）'])},
            {"role": "user", "content": f"Translate English to {target_column}: {str(data['English(译)'])}, please directly translate it and do not output any extra content"}
        ],
        temperature=0
    )
    return index, completion.choices[0].message.content

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, target_column] = output

# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx", index=False)

(0, 'matahari\n')

# 翻译123种语言

初翻

In [2]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"C:\Users\wangz\Downloads\20241203 123种语种翻译数据.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        tasks.append((row_index, column, text_to_translate))

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    messages = [
        {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
        {"role": "assistant", "content": str(data[column][1])},
        {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
    ]
    try:
        completion =  client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )
        translated_text = completion.choices[0].message.content
        # Write the translated text back to the DataFrame in a thread-safe manner
        with data_lock:
            data.at[row_index, column] = translated_text
        return (row_index, column, translated_text)
    except Exception as e:
        print(f"Error translating row {row_index}, column {column}: {e}")
        return (row_index, column, None)

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data.xlsx", index=False)

100%|██████████| 14640/14640 [02:03<00:00, 118.87it/s]


精翻

In [6]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"D:\Projects\ai-translator\src\translated_data.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        tasks.append((row_index, column, text_to_translate))

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    if text_to_translate.strip() == data.at[row_index, column].strip():
        messages = [
            {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
            {"role": "assistant", "content": str(data[column][1])},
            {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
        ]
        try:
            completion =  client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0
            )
            translated_text = completion.choices[0].message.content
            # Write the translated text back to the DataFrame in a thread-safe manner
            with data_lock:
                data.at[row_index, column] = translated_text
            return (row_index, column, translated_text)
        except Exception as e:
            print(f"Error translating row {row_index}, column {column}: {e}")
            return (row_index, column, None)
    else:
        return (row_index, column, data.at[row_index, column])

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data_1.xlsx", index=False)

100%|██████████| 14640/14640 [00:01<00:00, 14606.36it/s] 
